In [32]:
# セル1: 共通ベースクラス（BaseScraper）

"""
BaseScraper - スクレイピングの基本機能を提供する基底クラス

このモジュールはスクレイピングの共通機能を持つ基底クラスを提供します。
設定ファイルの読み込み、ブラウザの初期化、ログ機能などの共通機能を集約しています。
"""

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import os
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import logging
from dotenv import load_dotenv
from pathlib import Path
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
import yaml
import json


class BaseScraper:
    """
    スクレイピングの基本機能を提供する基底クラス
    
    このクラスはスクレイピングに必要な以下の共通機能を提供します:
    - 設定ファイルの読み込み
    - ブラウザの初期化と設定
    - ログ機能
    - CSVファイル操作
    """

    def _load_config(self, config_path=None):
        """
        設定ファイルを読み込みます
        
        Args:
            config_path (str): 設定ファイルのパス
        
        Returns:
            dict: 設定データ
        """
        # デフォルトのパス
        if config_path is None:
            config_path = os.path.join(self.root_dir, 'config', 'settings.yaml')
        
        print(f"設定ファイルパス: {config_path}")
        
        try:
            if not os.path.exists(config_path):
                raise FileNotFoundError(f"設定ファイルが見つかりません: {config_path}")
                
            # YAMLファイルを読み込む
            with open(config_path, 'r', encoding='utf-8') as f:
                config = yaml.safe_load(f)
            
            # 環境変数から認証情報を取得して設定ファイルにマージ
            self._merge_env_variables(config)
            
            print("設定ファイルを正常に読み込みました")
            return config
        except Exception as e:
            print(f"設定ファイルの読み込みエラー: {str(e)}")
            # エラーを上位に伝播させる
            raise

    def load_config(config_path=None):
        """
        設定ファイルを読み込み、環境変数で値を置き換えます
        
        Args:
            config_path (str): 設定ファイルのパス
        
        Returns:
            dict: 設定データ
        """
        # .envファイルを読み込み
        load_dotenv()
        
        # デフォルトのパス
        if config_path is None:
            # プロジェクトルートを特定
            current_dir = os.getcwd()
            project_root = current_dir
            
            # notebooksディレクトリにいる場合はルートに戻る
            if os.path.basename(current_dir) == 'notebooks':
                project_root = os.path.dirname(current_dir)
                
            config_path = os.path.join(project_root, "config", "settings.yaml")
        
        print(f"設定ファイルパス: {config_path}")
        
        try:
            if not os.path.exists(config_path):
                raise FileNotFoundError(f"設定ファイルが見つかりません: {config_path}")
                
            with open(config_path, 'r', encoding='utf-8') as f:
                # YAMLファイルを読み込む
                yaml_content = f.read()
                
                # 環境変数プレースホルダーを置換
                pattern = r'\$\{([A-Za-z0-9_]+)\}'
                
                def replace_env_var(match):
                    env_var = match.group(1)
                    return os.environ.get(env_var, f"${{{env_var}}}")
                
                processed_yaml = re.sub(pattern, replace_env_var, yaml_content)
                
                # 処理済みYAMLを解析
                config = yaml.safe_load(processed_yaml)
            
            print("設定ファイルを正常に読み込みました")
            return config
        except Exception as e:
            print(f"設定ファイルの読み込みエラー: {str(e)}")
            # エラーを上位に伝播させる
            raise

    def _merge_env_variables(self, config):
        """環境変数から認証情報を取得し、設定ファイルにマージする"""
        # このメソッドは子クラスでオーバーライドして実装します
        pass

    def setup_logging(self):
        """ログ機能のセットアップ"""
        # すでに存在するハンドラを削除（重複を防ぐため）
        for handler in logging.root.handlers[:]:
            logging.root.removeHandler(handler)
        
        # ログファイルパスの設定
        log_filename = f'{self.site_name}_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'
        log_file = os.path.join(self.log_dir, log_filename)
        
        # 基本設定
        logging.basicConfig(
            filename=log_file,
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            encoding='utf-8'
        )
        
        # コンソールにもログを出力
        console = logging.StreamHandler()
        console.setLevel(logging.INFO)
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        console.setFormatter(formatter)
        logging.getLogger('').addHandler(console)
        
        # ログファイルの場所を明示的に表示
        print(f"ログファイル出力先: {log_file}")
        logging.info(f"ログ機能の初期化が完了しました: {log_file}")

    def _find_project_root(self):
        """
        プロジェクトのルートディレクトリを検出する
        """
        # 現在のファイルの絶対パスを取得
        current_dir = os.path.abspath(os.getcwd())
        
        # 親ディレクトリを探索
        path = Path(current_dir)
        while True:
            # .gitディレクトリがあればそれをルートとみなす
            if (path / '.git').exists():
                return str(path)
            
            # プロジェクトのルートを示す他のファイル/ディレクトリの存在チェック
            if (path / 'setup.py').exists() or (path / 'README.md').exists():
                return str(path)
            
            # これ以上上の階層がない場合は現在のディレクトリを返す
            if path.parent == path:
                return str(path)
            
            # 親ディレクトリへ
            path = path.parent

    def _setup_browser(self):
        """
        Seleniumのブラウザを設定します
        
        ChromeOptionsの設定とWebDriverの起動を行います
        """
        # Chromeオプション設定
        chrome_options = Options()
        if self.headless_mode:
            chrome_options.add_argument("--headless")  # ヘッドレスモード（画面表示なし）
        chrome_options.add_argument("--disable-gpu")  # GPU無効化（ヘッドレス時に推奨）
        chrome_options.add_argument("--window-size=1920x1080")  # ウィンドウサイズ設定
        
        try:
            # ドライバーパスが指定されている場合は直接使用
            if hasattr(self, 'driver_path') and self.driver_path:
                self.browser = webdriver.Chrome(service=Service(self.driver_path), options=chrome_options)
            # 指定がなければWebDriverManagerを使用
            else:
                self.browser = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)
            
            self.wait = WebDriverWait(self.browser, 10)  # 要素が見つかるまで最大10秒待機
            print("ブラウザの初期化に成功しました")
        except Exception as e:
            print(f"ブラウザの初期化に失敗しました: {str(e)}")
            raise

    def _setup_session(self):
        """
        requestsのセッションを設定します
        
        Seleniumのブラウザから取得したCookieをrequestsセッションに設定します
        これによりログイン状態を維持したままBeautifulSoupでページ取得できます
        """
        # ブラウザのCookieを取得
        cookies = self.browser.get_cookies()
        
        # requestsセッションを作成
        self.session = requests.Session()
        
        # Seleniumから取得したCookieをrequestsセッションに追加
        for cookie in cookies:
            self.session.cookies.set(cookie['name'], cookie['value'])

    def prepare_csv(self):
        """
        CSVファイルを初期化します
        
        既存のファイルがある場合は削除し、新しいヘッダー付きのCSVを作成します
        """
        # 既存ファイルがあれば削除
        if os.path.exists(self.csv_filename):
            os.remove(self.csv_filename)
        
        # カラム構成でCSVを初期化
        df = pd.DataFrame(columns=self.columns)
        df.to_csv(self.csv_filename, index=False, encoding="utf-8-sig")
        
        print(f"CSVファイル {self.csv_filename} を初期化しました")

    def save_to_csv(self, data):
        """
        データをCSVファイルに保存します
        
        Args:
            data (list): 保存するデータ（リストのリスト形式）
            
        Returns:
            bool: 保存成功時はTrue、失敗時はFalse
        """
        try:
            if data:
                # データフレームに変換
                df = pd.DataFrame(data, columns=self.columns)
                
                # CSVに追記（ヘッダーなし）
                df.to_csv(self.csv_filename, mode="a", index=False, header=False, encoding="utf-8-sig")
                return True
            else:
                print("保存するデータがありません")
                return False
        except Exception as e:
            print(f"CSV保存エラー: {str(e)}")
            return False

    def __init__(self, site_name, config_path=None, headless_mode=False):
        """
        BaseScraperの初期化
        
        Args:
            site_name (str): スクレイピングするサイト名（ログファイル名などに使用）
            config_path (str): 設定ファイルのパス（指定しない場合はデフォルト値を使用）
            headless_mode (bool): ブラウザを画面に表示せずに実行する場合はTrue
        """
        # サイト名の設定
        self.site_name = site_name
        
        # プロジェクトルートディレクトリの検出
        self.root_dir = self._find_project_root()
        
        # 環境変数の読み込み
        load_dotenv(os.path.join(self.root_dir, '.env'))
        
        # ディレクトリパスの設定
        self.data_dir = os.path.join(self.root_dir, 'data')
        self.log_dir = os.path.join(self.root_dir, 'logs')
        
        # ディレクトリが存在しない場合は作成
        os.makedirs(self.data_dir, exist_ok=True)
        os.makedirs(self.log_dir, exist_ok=True)
        
        # 設定ファイルの読み込み
        self.config = self._load_config(config_path)
        
        # 基本設定
        self.headless_mode = headless_mode
        self.browser = None
        self.wait = None
        self.session = None
        
        # 実行時間計測用
        self.start_time = None
        self.end_time = None

        # ログ設定を初期化時に行う
        self.setup_logging()

    def close(self): 
        """
        ブラウザを閉じてリソースを解放します
        """
        if self.browser:
            self.browser.quit()
            print("ブラウザを閉じました")

In [34]:
# セル2: ヨリヤス価格検索クラス（YoriyasuPriceFinder）

"""
YoriyasuPriceFinder - ヨリヤスから商品の最安価格情報を取得するモジュール

このモジュールはJANコードリストに基づいて、ヨリヤスで各商品の最安値情報を取得します。
BaseScraper を継承し、ヨリヤス固有の機能を実装しています。
"""

class YoriyasuPriceFinder(BaseScraper):
    """
    ヨリヤスWebサイトから商品の最安値情報を取得するクラス
    
    JANコードのリストを入力として、各商品の最安値情報（価格、送料、販売サイト等）を取得します。
    """
    
    def _merge_env_variables(self, config):
        """環境変数から設定情報を取得し、設定ファイルにマージする"""
        # ヨリヤスはログイン不要なのでその部分は省略
        
        # デフォルトの出力設定（なければ設定）
        if 'price_finder' not in config:
            config['price_finder'] = {}
        if 'yoriyasu' not in config['price_finder']:
            config['price_finder']['yoriyasu'] = {}
        if 'output' not in config['price_finder']['yoriyasu']:
            config['price_finder']['yoriyasu']['output'] = {
                'input_file': 'jan_list.csv',  # JANコードリストの入力ファイル
                'output_file': 'yoriyasu_prices.csv',  # 出力ファイル
                'log_dir': 'logs'
            }
    
    def __init__(self, config_path=None, headless_mode=False):
        """
        YoriyasuPriceFinderの初期化
        
        Args:
            config_path (str): 設定ファイルのパス（指定しない場合はデフォルト値を使用）
            headless_mode (bool): ブラウザを画面に表示せずに実行する場合はTrue
        """
        # 親クラス(BaseScraper)の初期化
        super().__init__('yoriyasu_price', config_path, headless_mode)
        
        # ヨリヤス固有の設定
        self.yoriyasu_config = self.config['price_finder']['yoriyasu']
        self.base_url = "https://yoriyasu.jp/products"
        
        # 出力設定（設定ファイルから読み込み）
        output_config = self.yoriyasu_config.get('output', {})
        self.input_file = os.path.join(self.data_dir, output_config.get('input_file', 'jan_list.csv'))
        csv_filename = output_config.get('output_file', 'yoriyasu_prices.csv')
        
        # CSVのフルパスを設定
        self.csv_filename = os.path.join(self.data_dir, csv_filename)
        print(f"入力ファイル: {self.input_file}")
        print(f"CSVファイル出力先: {self.csv_filename}")
        
        # 1行に複数サイト情報を含めるように変更
        # 4サイト分のカラムを用意
        self.max_sites = 4  # 保存する最大サイト数
        
        # カラム設定（水平方向に拡張）
        self.columns = ["JANコード"]
        for i in range(1, self.max_sites + 1):
            self.columns.extend([
                f"商品名{i}", f"仕入価格{i}", f"送料{i}", 
                f"出品者名{i}", f"仕入れサイト{i}", f"仕入れURL{i}"
            ])
        
        # ブラウザ設定
        self._setup_browser()
        
        # 待機時間のランダム化用
        import random
        self.random = random

        # 監視用のカウンター
        self.total_requests = 0
        self.failed_requests = 0
        self.status_codes = {}
        self.avg_response_time = 0

        self.last_request_time = None
        self.request_intervals = []
        
    
    def load_jan_codes(self):
        """
        JANコードリストを読み込みます
        
        Returns:
            list: JANコードのリスト
        """
        jan_codes = []
        
        try:
            # ファイルの存在確認
            if not os.path.exists(self.input_file):
                raise FileNotFoundError(f"JANコードリストファイルが見つかりません: {self.input_file}")
            
            # CSVからJANコードを読み込む
            import csv
            with open(self.input_file, 'r', encoding='utf-8-sig') as f:
                reader = csv.reader(f)
                # ヘッダー行があるかチェック
                header = next(reader, None)
                
                # ヘッダー行があれば、JANコードのカラムを探す
                jan_column_index = 0
                if header:
                    for i, col in enumerate(header):
                        if 'JAN' in col.upper() or 'コード' in col or 'CODE' in col.upper():
                            jan_column_index = i
                            break
                
                # JANコードを読み込む
                for row in reader:
                    if row and len(row) > jan_column_index:
                        jan_code = row[jan_column_index].strip()
                        if jan_code and jan_code.isdigit() and len(jan_code) >= 8:  # 有効なJANコードのみ
                            jan_codes.append(jan_code)
            
            print(f"{len(jan_codes)}件のJANコードを読み込みました")
            return jan_codes
            
        except Exception as e:
            print(f"JANコード読み込みエラー: {str(e)}")
            logging.error(f"JANコード読み込みエラー: {str(e)}")
            return []

    def get_price_info(self, jan_code):
        """
        指定されたJANコードで検索し、最安値情報を取得します
        
        Args:
            jan_code (str): 検索するJANコード
            
        Returns:
            list: 商品の最安値情報のリスト（1行に複数サイト情報）
        """
        price_data_row = [jan_code]  # 1行のデータ（JANコードから始まる）
        search_url = f"{self.base_url}?sort=priceLow&page=1&keyword={jan_code}"
        print(f"JANコード {jan_code} の最安値検索中... ({search_url})")
        
        try:
            # 検索ページにアクセス
            self.browser.get(search_url)
            
            # より確実な待機 - ページ全体が読み込まれるまで待機（最大15秒） 「EC.visibility_of_element_located」があるのでコメントアウト
            # WebDriverWait(self.browser, 15).until(
            #     EC.presence_of_element_located((By.TAG_NAME, "body"))
            # )

            WebDriverWait(self.browser, 15).until(
                EC.visibility_of_element_located((By.CLASS_NAME, "styles_priceInfo__kzr_s"))
            ) # サイト情報ボックスの要素が表示されるまで待機
            
            # # 追加の待機時間（JavaScriptの実行を待つ） 「EC.visibility_of_element_located」があるのでコメントアウト
            # time.sleep(0.3)
            
            # ページのHTMLを取得
            html_content = self.browser.page_source
            print(f"ページHTMLを取得しました（長さ: {len(html_content)}文字）")
            
            # BeautifulSoupでHTMLを解析
            soup = BeautifulSoup(html_content, 'html.parser')
            
            # 「0件」かどうかチェック
            no_results = soup.find(string=lambda s: "（0件）" in s if s else False)
            if no_results:
                print(f"JANコード {jan_code} の商品は見つかりませんでした（0件）")
                
                # 「0件」の場合はエラーとしてカウント
                self.total_requests += 1
                self.failed_requests += 1
                
                # 「0件」の場合は通常より長めに待機（2秒）
                print("「0件」結果のため、追加で2秒待機します...")
                time.sleep(2.0)
                
                return []
            
            # 商品リストを取得 - style属性で特定する
            product_stacks = soup.find_all("div", style=lambda s: "width: 100%; gap: calc(0.425rem); position: relative;" in s if s else False)
            
            # 商品リストが見つからない場合、別の方法も試す
            if not product_stacks:
                # クラス名の部分一致でも試してみる
                product_stacks = soup.find_all(class_=lambda c: "styles_stack__" in c if c else False)
                print(f"style属性での検索失敗、クラス名部分一致で {len(product_stacks)} 件見つかりました")
            
            # 商品リストが見つからない場合
            if not product_stacks:
                print(f"JANコード {jan_code} の商品リストが見つかりませんでした")
                return []
            
            # 商品スタックを選択（最初のスタックが通常は商品リスト）
            product_stack = product_stacks[0]
            
            # 各商品のaタグを取得 - 直接の子要素として探す
            product_links = product_stack.find_all("a", href=True, recursive=True)
            
            print(f"商品リンク数: {len(product_links)}")
            
            if not product_links:
                # 別の方法でaタグを探す - 階層を気にせず全て探す
                product_links = soup.find_all("a", href=lambda h: "amazon.co.jp" in h or 
                                                               "rakuten.co.jp" in h or 
                                                               "yahoo.co.jp" in h or 
                                                               "wowma.jp" in h or
                                                               "valuecommerce.com" in h or
                                                               "linksynergy.com" in h
                                              if h else False)
                print(f"別の方法で見つけた商品リンク数: {len(product_links)}")
                
                if not product_links:
                    print(f"JANコード {jan_code} の商品リンクが見つかりませんでした")
                    return []
            
            # 最大サイト数に制限
            product_links = product_links[:self.max_sites]
            site_count = 0
            
            # 各商品の情報を抽出
            for product_link in product_links:
                try:
                    # サイト番号（1から始まる）
                    site_count += 1
                    
                    # 商品URL - アフィリエイトリンクの場合は対応
                    raw_url = product_link["href"]
                    
                    # 実際の商品URLを抽出（アフィリエイトリンクの場合）
                    if "amazon.co.jp" in raw_url:
                        # Amazonのアフィリエイトリンク処理
                        import re
                        amazon_id_match = re.search(r'dp/([A-Za-z0-9]+)', raw_url)
                        if amazon_id_match:
                            product_id = amazon_id_match.group(1)
                            # 複数商品表示用のリンクに変換
                            product_url = f"https://www.amazon.co.jp/gp/offer-listing/{product_id}"
                        else:
                            product_url = raw_url
                    elif "valuecommerce.com" in raw_url and "vc_url=" in raw_url:
                        # ValueCommerceのアフィリエイトリンク（ヤフーショッピングやLOHACO）
                        import urllib.parse
                        vc_url_param = re.search(r'vc_url=([^&]+)', raw_url)
                        if vc_url_param:
                            product_url = urllib.parse.unquote(vc_url_param.group(1))
                        else:
                            product_url = raw_url
                    elif "rakuten.co.jp" in raw_url or "hb.afl.rakuten.co.jp" in raw_url:
                        # 楽天のアフィリエイトリンク
                        import urllib.parse
                        pc_param = re.search(r'pc=([^&]+)', raw_url)
                        if pc_param:
                            product_url = urllib.parse.unquote(pc_param.group(1))
                        else:
                            # 別の形式の楽天リンクの可能性も確認
                            rakuten_item_match = re.search(r'(https://item.rakuten.co.jp/[^/]+/[^/&?]+)', raw_url)
                            if rakuten_item_match:
                                product_url = rakuten_item_match.group(1)
                            else:
                                product_url = raw_url
                    elif "linksynergy.com" in raw_url and "murl=" in raw_url:
                        # LinkSynergyのアフィリエイトリンク（auPAY等）
                        import urllib.parse
                        murl_param = re.search(r'murl=([^&]+)', raw_url)
                        if murl_param:
                            product_url = urllib.parse.unquote(murl_param.group(1))
                        else:
                            product_url = raw_url
                    else:
                        # その他のリンク（通常のURLまたは未知のアフィリエイトリンク）
                        product_url = raw_url
                    
                    print(f"商品URL{site_count}: {product_url}")
                    
                    # 商品名 - 複数の可能性のあるクラスで探す
                    name_element = product_link.find(class_=lambda c: "saleName" in c if c else False) or \
                                   product_link.find("p")  # フォールバック: 最初のp要素
                    
                    if not name_element:
                        print(f"商品名要素{site_count}が見つかりません")
                        # 空の値でデータを追加（行の構造を維持するため）
                        price_data_row.extend(["", "", "", "", "", ""])
                        continue
                        
                    product_name = name_element.text.strip()
                    print(f"商品名{site_count}: {product_name}")
                    
                    # 価格情報を含む要素 - flexスタイルで探す（複数の方法で試行）
                    price_container = product_link.find("div", style=lambda s: "flex-basis: 30%" in s if s else False) or \
                                     product_link.find("div", style=lambda s: "flex-grow: 1" in s if s else False)
                    
                    if not price_container:
                        print(f"価格コンテナ{site_count}が見つかりません")
                        # 空の値でデータを追加（行の構造を維持するため）
                        price_data_row.extend([product_name, "", "", "", "", product_url])
                        continue
                    
                    # 送料情報 - クラス名で探す（複数の可能性を試す）
                    shipping_element = price_container.find(class_=lambda c: "shipping" in c.lower() if c else False) or \
                                      price_container.find("span")  # フォールバック: 最初のspan
                    shipping = shipping_element.text.strip() if shipping_element else "不明"
                    print(f"送料{site_count}: {shipping}")
                    
                    # 価格情報 - クラス名で探す（複数の可能性を試す）
                    price_element = price_container.find(class_=lambda c: "price" in c.lower() if c else False) or \
                                   price_container.find("span", recursive=True)  # フォールバック
                    
                    price_value = ""
                    if price_element:
                        # 価格から数値のみを抽出
                        price_text = price_element.text.strip()
                        price_match = re.search(r'[¥￥]\s*([\d,]+)', price_text)
                        price_value = price_match.group(1).replace(',', '') if price_match else ""
                        print(f"価格{site_count}: {price_value}")
                    
                    # 出品者とサイト情報を含む要素 - スタイルで探す
                    shop_container = product_link.find("div", style=lambda s: "flex-grow: 999" in s if s else False) or \
                                    product_link.find("div", style=lambda s: "min-width: 70%" in s if s else False)
                    
                    shop_name = "不明"
                    site_name = "不明"
                    
                    if shop_container:
                        # 出品者情報のコンテナ
                        shop_info = shop_container.find(class_=lambda c: "shopInfo" in c if c else False) or \
                                   shop_container.find("div")  # フォールバック
                        
                        if shop_info:
                            # 出品者名
                            shop_name_element = shop_info.find(class_=lambda c: "shopName" in c if c else False) or \
                                              shop_info.find("span")  # フォールバック
                            shop_name = shop_name_element.text.strip() if shop_name_element else "不明"
                            
                            # 仕入れサイト（画像のalt属性）
                            site_image = shop_info.find("img")
                            site_name = site_image.get("alt", "不明") if site_image else "不明"
                    
                    print(f"出品者名{site_count}: {shop_name}, 仕入れサイト{site_count}: {site_name}")
                    
                    # データ行に追加
                    price_data_row.extend([
                        product_name,    # 商品名
                        price_value,     # 仕入価格
                        shipping,        # 送料
                        shop_name,       # 出品者名
                        site_name,       # 仕入れサイト
                        product_url      # 仕入れURL
                    ])
                    
                    print(f"サイト{site_count}のデータを追加しました")
                    
                except Exception as e:
                    print(f"サイト{site_count}の情報抽出エラー: {str(e)}")
                    logging.error(f"サイト{site_count}の情報抽出エラー: {str(e)}")
                    logging.error(traceback.format_exc())
                    
                    # エラーが発生した場合も、行の構造を維持するために空の値を追加
                    price_data_row.extend(["", "", "", "", "", ""])
                    continue
            
            # 不足している分のサイト情報を空データで埋める
            while site_count < self.max_sites:
                site_count += 1
                price_data_row.extend(["", "", "", "", "", ""])
            
            # 1行分のデータを返す（1JANコードに対して1行）
            if len(price_data_row) > 1:  # JANコード以外のデータがある場合
                return [price_data_row]
            
        except Exception as e:
            print(f"検索ページアクセスエラー (JANコード {jan_code}): {str(e)}")
            logging.error(f"検索ページアクセスエラー (JANコード {jan_code}): {str(e)}")
            logging.error(traceback.format_exc())  # 詳細なエラーログ
        
        return []


    def monitor_request(self, url):
        """
        HTTPリクエストを監視し、ステータスコード、レスポンスタイム、エラーをチェックする
        
        Args:
            url (str): リクエスト先のURL
            
        Returns:
            tuple: (成功したかどうか, ステータスコード, レスポンスタイム)
        """
        import time
        import requests
        
        start_time = time.time()
        try:
            # Seleniumのブラウザは既にURLにアクセスしているため
            # 別途requestsで直接アクセスして状態をチェック
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, timeout=10)
            status_code = response.status_code
            success = 200 <= status_code < 400
        except Exception as e:
            logging.error(f"リクエスト監視エラー: {str(e)}")
            status_code = 0
            success = False
        
        response_time = time.time() - start_time
        
        # 結果をログに記録
        log_msg = f"URL: {url}, ステータス: {status_code}, 応答時間: {response_time:.2f}秒, 成功: {success}"
        if success:
            logging.info(log_msg)
        else:
            logging.warning(log_msg)
        
        return success, status_code, response_time
    
    

    def check_error_rate(self):
        """
        エラー率を計算し、高すぎる場合は警告またはスクレイピングを一時停止する
        
        Returns:
            bool: 続行可能な場合はTrue、一時停止すべき場合はFalse
        """
        if self.total_requests == 0:
            return True
        
        error_rate = self.failed_requests / self.total_requests
        
        # エラー率をログに記録
        logging.info(f"現在のリクエスト数: {self.total_requests}, エラー数: {self.failed_requests}, エラー率: {error_rate:.2%}")
        logging.info(f"ステータスコード分布: {self.status_codes}")
        logging.info(f"平均応答時間: {self.avg_response_time:.2f}秒")
        
        # エラー率が30%を超える場合は警告
        if error_rate > 0.3:
            logging.warning(f"エラー率が高すぎます ({error_rate:.2%}). IPBANの可能性があります。")
            
            # エラー率が50%を超える場合は一時停止を推奨
            if error_rate > 0.5:
                logging.error("エラー率が50%を超えました。スクレイピングを一時停止します。")
                return False
        
        return True


    def refresh_session(self):
        """
        ブラウザのセッションをリフレッシュし、新しいCookieを取得する
        """
        logging.info("ブラウザセッションをリフレッシュします...")
        
        try:
            # 現在のブラウザを閉じる
            if self.browser:
                self.browser.quit()
            
            # 新しいブラウザを起動
            self._setup_browser()
            
            # ホームページにアクセスして新しいCookieを取得
            self.browser.get("https://yoriyasu.jp/")
            time.sleep(2)
            
            # リクエスト関連の統計をリセット
            self.total_requests = 0
            self.failed_requests = 0
            self.status_codes = {}
            self.avg_response_time = 0
            
            logging.info("ブラウザセッションのリフレッシュに成功しました")
            return True
        except Exception as e:
            logging.error(f"セッションリフレッシュエラー: {str(e)}")
            return False
    
    
    def find_all_prices(self):
        """
        すべてのJANコードに対して最安値情報を検索します
        
        Returns:
            int: 取得した商品データの行数
        """
        # 実行時間測定開始
        self.start_time = time.time()
        
        # CSVを初期化
        self.prepare_csv()
        
        # JANコードの読み込み
        jan_codes = self.load_jan_codes()
        if not jan_codes:
            print("有効なJANコードがありません。処理を終了します。")
            return 0
        
        # 各JANコードの処理
        total_rows = 0
        request_count = 0  # リクエスト数のカウンター
        
        # 進捗表示用
        total_jan_codes = len(jan_codes)
        
        for i, jan_code in enumerate(jan_codes, 1):
            print(f"\n----- JANコード {jan_code} の処理中 ({i}/{total_jan_codes}) -----")

            # リクエスト間隔を測定
            current_time = time.time()
            if self.last_request_time is not None:
                interval = current_time - self.last_request_time
                self.request_intervals.append(interval)
                avg_interval = sum(self.request_intervals) / len(self.request_intervals)
                print(f"前回のリクエストから {interval:.2f}秒、平均間隔: {avg_interval:.2f}秒")
            self.last_request_time = current_time
            
            
            # 100件ごとにセッションをリフレッシュ
            request_count += 1
            if request_count > 50:
                print("50件のリクエスト完了。セッションをリフレッシュします...")
                self.refresh_session()
                request_count = 0
            
            # 最安値情報の取得（1行のデータを取得）
            price_data = self.get_price_info(jan_code)
            
            # エラー率をチェック
            if not self.check_error_rate():
                print("エラー率が高すぎるため、一時停止します。15分後に再開してください。")
                logging.error("エラー率が閾値を超えたため、スクレイピングを中断します。")
                break
            
            # データを保存
            if price_data:
                self.save_to_csv(price_data)
                total_rows += len(price_data)
                print(f"JANコード {jan_code} の最安値データを保存しました")
            
            # 連続アクセスによるブロック防止のための待機（少しランダム化）
            if i < total_jan_codes:
                wait_time = 0.0 + self.random.random() * 0.4 
                print(f"{wait_time:.2f}秒待機中...")
                time.sleep(wait_time)
        
        # 実行時間測定終了
        self.end_time = time.time()
        elapsed_time = self.end_time - self.start_time
        
        print(f"\n===== 最安値検索完了 - 合計 {total_rows} 件 ====")
        print(f"実行時間: {elapsed_time:.2f} 秒")
        
        # リクエスト統計
        if self.request_intervals:
            avg_interval = sum(self.request_intervals) / len(self.request_intervals)
            min_interval = min(self.request_intervals)
            max_interval = max(self.request_intervals)
            print(f"リクエスト間隔: 平均 {avg_interval:.2f}秒 (最小 {min_interval:.2f}秒、最大 {max_interval:.2f}秒)")
            print(f"リクエスト数: {len(self.request_intervals) + 1}")
        
        return total_rows

In [36]:
# セル3: 価格検索のインスタンスを作成して実行するコード
if __name__ == "__main__":
    import traceback  # 詳細なエラーログ用
    
    # 価格検索のインスタンスを作成
    price_finder = YoriyasuPriceFinder(headless_mode=False)

    try:
        # JANコードリストに基づいて全商品の最安値情報を検索
        price_finder.find_all_prices()
    finally:
        # 終了処理
        price_finder.close()

2025-03-07 00:12:06,031 - INFO - ログ機能の初期化が完了しました: C:\Users\inato\Documents\amazon-research\logs\yoriyasu_price_20250307_001206.log
2025-03-07 00:12:06,032 - INFO - ====== WebDriver manager ======


設定ファイルパス: C:\Users\inato\Documents\amazon-research\config\settings.yaml
設定ファイルを正常に読み込みました
ログファイル出力先: C:\Users\inato\Documents\amazon-research\logs\yoriyasu_price_20250307_001206.log
入力ファイル: C:\Users\inato\Documents\amazon-research\data\jan_list.csv
CSVファイル出力先: C:\Users\inato\Documents\amazon-research\data\yoriyasu_prices.csv


2025-03-07 00:12:07,425 - INFO - Get LATEST chromedriver version for google-chrome
2025-03-07 00:12:07,514 - INFO - Get LATEST chromedriver version for google-chrome
2025-03-07 00:12:07,543 - INFO - Driver [C:\Users\inato\.wdm\drivers\chromedriver\win64\133.0.6943.141\chromedriver-win32/chromedriver.exe] found in cache


ブラウザの初期化に成功しました
CSVファイル C:\Users\inato\Documents\amazon-research\data\yoriyasu_prices.csv を初期化しました
1000件のJANコードを読み込みました

----- JANコード 4901987230748 の処理中 (1/1000) -----
JANコード 4901987230748 の最安値検索中... (https://yoriyasu.jp/products?sort=priceLow&page=1&keyword=4901987230748)
ページHTMLを取得しました（長さ: 153456文字）
商品リンク数: 39
商品URL1: https://www.amazon.co.jp/gp/offer-listing/B0154K8X0E
商品名1: 東洋アルミ(Toyo Aluminium) 換気扇 フィルター 整流板 貼るだけ 切り取りミシン目付き 約64cm×91cm 1枚入 フィルたん S3074
送料1: 送料無料
価格1: 565
出品者名1: Amazon.co.jp, 仕入れサイト1: AMAZON
サイト1のデータを追加しました
商品URL2: https://store.shopping.yahoo.co.jp/ladydrugheartshop/4901987230748.html
商品名2: 東洋アルミ　整流板付専用パッと貼るだけスーパーフィルター
送料2: 条件付き送料無料
価格2: 527
出品者名2: くすりのレデイハートショップ, 仕入れサイト2: YAHOO
サイト2のデータを追加しました
商品URL3: https://item.rakuten.co.jp/ladydrug/4901987230748/
商品名3: 東洋アルミ　整流板付専用パッと貼るだけスーパーフィルター
送料3: 送料別
価格3: 512
出品者名3: くすりのレデイハートショップ, 仕入れサイト3: RAKUTEN
サイト3のデータを追加しました
商品URL4: https://wowma.jp/item/711510973
商品名4: 東洋アルミ パッと貼るだけ 整流板付専用 スーパーフィルター 3074 1枚入(4901987230748)
送料4: 送料

2025-03-07 00:13:10,450 - INFO - ブラウザセッションをリフレッシュします...


...

ページHTMLを取得しました（長さ: 120680文字）
商品リンク数: 25
サイト1の情報抽出エラー: cannot access local variable 're' where it is not associated with a value
サイト2の情報抽出エラー: cannot access local variable 're' where it is not associated with a value
サイト3の情報抽出エラー: cannot access local variable 're' where it is not associated with a value
サイト4の情報抽出エラー: cannot access local variable 're' where it is not associated with a value
JANコード 4901105310642 の最安値データを保存しました
0.13秒待機中...

----- JANコード 4979508585028 の処理中 (996/1000) -----
前回のリクエストから 1.43秒、平均間隔: 1.75秒
JANコード 4979508585028 の最安値検索中... (https://yoriyasu.jp/products?sort=priceLow&page=1&keyword=4979508585028)


2025-03-07 00:41:07,281 - ERROR - サイト1の情報抽出エラー: cannot access local variable 're' where it is not associated with a value
2025-03-07 00:41:07,281 - ERROR - Traceback (most recent call last):
  File "C:\Users\inato\AppData\Local\Temp\ipykernel_28272\1358825377.py", line 247, in get_price_info
    vc_url_param = re.search(r'vc_url=([^&]+)', raw_url)
                   ^^
UnboundLocalError: cannot access local variable 're' where it is not associated with a value

2025-03-07 00:41:07,281 - ERROR - サイト2の情報抽出エラー: cannot access local variable 're' where it is not associated with a value
2025-03-07 00:41:07,281 - ERROR - Traceback (most recent call last):
  File "C:\Users\inato\AppData\Local\Temp\ipykernel_28272\1358825377.py", line 255, in get_price_info
    pc_param = re.search(r'pc=([^&]+)', raw_url)
               ^^
UnboundLocalError: cannot access local variable 're' where it is not associated with a value

2025-03-07 00:41:07,281 - ERROR - サイト3の情報抽出エラー: cannot access local variable 're

ページHTMLを取得しました（長さ: 131766文字）
商品リンク数: 30
サイト1の情報抽出エラー: cannot access local variable 're' where it is not associated with a value
サイト2の情報抽出エラー: cannot access local variable 're' where it is not associated with a value
サイト3の情報抽出エラー: cannot access local variable 're' where it is not associated with a value
サイト4の情報抽出エラー: cannot access local variable 're' where it is not associated with a value
JANコード 4979508585028 の最安値データを保存しました
0.01秒待機中...

----- JANコード 4901126526183 の処理中 (997/1000) -----
前回のリクエストから 1.33秒、平均間隔: 1.75秒
JANコード 4901126526183 の最安値検索中... (https://yoriyasu.jp/products?sort=priceLow&page=1&keyword=4901126526183)
ページHTMLを取得しました（長さ: 121287文字）
商品リンク数: 25
商品URL1: https://www.amazon.co.jp/gp/offer-listing/B0D1735GBX
商品名1: 岩崎工業 フェローズピッチャーシリーズ タテヨコ・スライドピッチャー 1.3 W 1ケース 12個×3小箱入
送料1: 送料別
価格1: 45976
出品者名1: A1 ショップ【3300円以上で全国送料無料】※一部配送料無料もございます, 仕入れサイト1: AMAZON
サイト1のデータを追加しました
商品URL2: https://store.shopping.yahoo.co.jp/k-mori/4901126526183.html
商品名2: 岩崎工業 タテヨコ・スライドピッチャー1.3 ホワイト K-1261 W キッチ

2025-03-07 00:41:14,538 - ERROR - サイト1の情報抽出エラー: cannot access local variable 're' where it is not associated with a value
2025-03-07 00:41:14,538 - ERROR - Traceback (most recent call last):
  File "C:\Users\inato\AppData\Local\Temp\ipykernel_28272\1358825377.py", line 247, in get_price_info
    vc_url_param = re.search(r'vc_url=([^&]+)', raw_url)
                   ^^
UnboundLocalError: cannot access local variable 're' where it is not associated with a value

2025-03-07 00:41:14,538 - ERROR - サイト2の情報抽出エラー: cannot access local variable 're' where it is not associated with a value
2025-03-07 00:41:14,538 - ERROR - Traceback (most recent call last):
  File "C:\Users\inato\AppData\Local\Temp\ipykernel_28272\1358825377.py", line 255, in get_price_info
    pc_param = re.search(r'pc=([^&]+)', raw_url)
               ^^
UnboundLocalError: cannot access local variable 're' where it is not associated with a value

2025-03-07 00:41:14,538 - ERROR - サイト3の情報抽出エラー: cannot access local variable 're

商品リンク数: 27
サイト1の情報抽出エラー: cannot access local variable 're' where it is not associated with a value
サイト2の情報抽出エラー: cannot access local variable 're' where it is not associated with a value
サイト3の情報抽出エラー: cannot access local variable 're' where it is not associated with a value
サイト4の情報抽出エラー: cannot access local variable 're' where it is not associated with a value
JANコード 4979508585059 の最安値データを保存しました
0.26秒待機中...

----- JANコード 4904705162734 の処理中 (999/1000) -----
前回のリクエストから 5.94秒、平均間隔: 1.75秒
JANコード 4904705162734 の最安値検索中... (https://yoriyasu.jp/products?sort=priceLow&page=1&keyword=4904705162734)
ページHTMLを取得しました（長さ: 129655文字）
商品リンク数: 29
商品URL1: https://www.amazon.co.jp/gp/offer-listing/B012MS3GBC
商品名1: トルネ エコバッグ コンビニバッグ 大 スパイス ブラウン P-3239
送料1: 送料無料
価格1: 734
出品者名1: Amazon.co.jp, 仕入れサイト1: AMAZON
サイト1のデータを追加しました
商品URL2: https://store.shopping.yahoo.co.jp/murauchi3/4904705162734.html
商品名2: TORUNE トルネ コンビニバッグ 大 スパイス (BR) P-3239
送料2: 送料別
価格2: 378
出品者名2: NEXT!, 仕入れサイト2: YAHOO
サイト2のデータを追加しました
商品URL3: h

In [ ]:
# # セル4: ヨリヤスのページ構造をデバッグするコード メモ：127件目から☓ 固定＋0.7~1.2秒

# def debug_yoriyasu_page_structure():
#     # ブラウザをセットアップ
#     from selenium import webdriver
#     from selenium.webdriver.chrome.service import Service
#     from selenium.webdriver.chrome.options import Options
#     from selenium.webdriver.common.by import By
#     from selenium.webdriver.support.ui import WebDriverWait
#     from selenium.webdriver.support import expected_conditions as EC
#     from webdriver_manager.chrome import ChromeDriverManager
#     from selenium.webdriver.chrome.service import Service as ChromeService
#     import time
#     from bs4 import BeautifulSoup
    
#     # Chromeを起動
#     chrome_options = Options()
#     browser = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)
    
#     try:
#         # テスト用のJANコード
#         jan_code = "4987601012921"  # サンプルで使用したコード
#         search_url = f"https://yoriyasu.jp/products?sort=priceLow&page=1&keyword={jan_code}"
        
#         print(f"URLにアクセス中: {search_url}")
#         browser.get(search_url)
        
#         # ページ読み込み待機
#         WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
#         time.sleep(2)
        
#         # HTMLを取得
#         html_content = browser.page_source
#         print(f"ページHTMLを取得しました（長さ: {len(html_content)}文字）")
        
#         # BeautifulSoupで解析
#         soup = BeautifulSoup(html_content, 'html.parser')
        
#         # ページ構造を分析
#         print("\n===== ページ構造分析 =====")
        
#         # メインコンテナ要素を探す
#         print("\n--- 1. メインコンテナ要素 ---")
#         main_containers = soup.find_all('div', class_=lambda c: c and ('container' in c.lower() or 'content' in c.lower()))
#         print(f"メインコンテナ候補数: {len(main_containers)}")
        
#         # product_stacksの探索（元のコードで使用されている）
#         print("\n--- 2. product_stacks要素（元のコードで探索している要素） ---")
#         product_stacks = soup.find_all(class_=lambda c: c and "styles_stack__vgYxm" in c)
#         print(f"product_stacks数: {len(product_stacks)}")
#         if product_stacks:
#             print(f"最初のproduct_stack: {product_stacks[0].name}, クラス: {product_stacks[0].get('class')}")
        
#         # 商品リストを含む可能性のある要素を探索
#         print("\n--- 3. 商品リスト要素 ---")
#         product_containers = soup.find_all(['div', 'ul'], class_=lambda c: c and ('product' in c.lower() or 'item' in c.lower() or 'list' in c.lower()))
#         print(f"商品リストコンテナ候補数: {len(product_containers)}")
        
#         # aタグの探索（商品リンク）
#         print("\n--- 4. 商品リンク（aタグ） ---")
#         # 階層構造を無視してすべてのaタグを探す
#         all_links = soup.find_all('a', href=True)
#         print(f"すべてのaタグ数: {len(all_links)}")
        
#         # 商品リンクと思われるものを抽出（通販サイトドメインを含むもの）
#         product_links = soup.find_all("a", href=lambda h: h and (
#             "amazon.co.jp" in h or 
#             "rakuten.co.jp" in h or 
#             "yahoo.co.jp" in h or 
#             "wowma.jp" in h or
#             "valuecommerce.com" in h or
#             "linksynergy.com" in h
#         ))
#         print(f"商品リンクと思われるaタグ数: {len(product_links)}")
        
#         # 最初の数件の商品リンクを表示
#         print("\n--- 5. 商品リンクサンプル ---")
#         for i, link in enumerate(product_links[:3]):
#             print(f"リンク {i+1}:")
#             print(f"  href: {link['href']}")
#             print(f"  親要素: {link.parent.name}, 親クラス: {link.parent.get('class')}")
#             print(f"  祖父要素: {link.parent.parent.name}, 祖父クラス: {link.parent.parent.get('class')}")
        
#         # ページの全クラス名を収集して分析
#         print("\n--- 6. ページ内のクラス名分析 ---")
#         all_classes = set()
#         for tag in soup.find_all(class_=True):
#             if tag.get('class'):
#                 all_classes.update(tag.get('class'))
        
#         print(f"ページ内の一意なクラス名数: {len(all_classes)}")
#         print("'stack'を含むクラス名:")
#         stack_classes = [cls for cls in all_classes if 'stack' in cls.lower()]
#         for cls in stack_classes:
#             print(f"  - {cls}")
        
#         print("\n'product'を含むクラス名:")
#         product_classes = [cls for cls in all_classes if 'product' in cls.lower()]
#         for cls in product_classes:
#             print(f"  - {cls}")
        
#         print("\n'item'を含むクラス名:")
#         item_classes = [cls for cls in all_classes if 'item' in cls.lower()]
#         for cls in item_classes:
#             print(f"  - {cls}")
        
#         # 商品コンテナの正確な構造を見つける
#         print("\n--- 7. 商品コンテナ構造の調査 ---")
#         # 様々なセレクタ方法を試す
#         selectors_to_try = [
#             "div.product-list", "div.item-list", "ul.products", 
#             "div.styles_stack__", "div[class*='stack']", 
#             "div[class*='product']", "div[class*='item']"
#         ]
        
#         for selector in selectors_to_try:
#             try:
#                 elements = soup.select(selector)
#                 print(f"セレクタ '{selector}' で {len(elements)} 個の要素を検出")
#                 if elements and len(elements) > 0:
#                     print(f"  最初の要素のクラス: {elements[0].get('class')}")
#                     print(f"  子要素数: {len(list(elements[0].children))}")
#             except Exception as e:
#                 print(f"セレクタ '{selector}' でエラー: {str(e)}")
        
#         # 解決策の提案
#         print("\n===== 推奨解決策 =====")
#         print("1. 正確なクラス名を使って商品コンテナを特定する")
#         print("2. product_stacksの検索を修正する:")
#         print("   - 完全一致ではなく部分一致を使う: class_=lambda c: c and 'stack' in c")
#         print("   - 特定のクラス名パターンを探す: div[class*='product'] または div[class*='item']")
#         print("3. 階層を気にせずリンクを直接探す方法を最初から使用する")
        
#     except Exception as e:
#         print(f"デバッグ中にエラーが発生しました: {str(e)}")
#     finally:
#         # ブラウザを閉じる
#         browser.quit()
#         print("デバッグが完了しました")

# # デバッグ実行
# debug_yoriyasu_page_structure()# a